<a href="https://colab.research.google.com/github/Iqra9999/Question-Answer-Generation-Pipeline-with-LLM/blob/main/QA_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#First of all, mounting google drive to access input files and save output files in google drive
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


# Preprocessing of news articles file

In [ ]:
import pandas as pd
import csv
import re

#The file contained various special characters in different formats, so everything is removed except the alphabets, integers and important characters.
def clean_text(text):
    #Removing words with extra spaces around
    text = re.sub(r'\s{2,}', ' ', text)

    #Regular expression for characters that are allowed
    pattern = r'[^a-zA-Z0-9,\$\(\):\;\. ]'

    #Replace the special characters with empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

#Path of input file
input_csv_file = '/content/drive/My Drive/news_dataset.csv'

#Reading the csv file in a df
df = pd.read_csv(input_csv_file)

#Rows with missing values are removed
df.dropna(inplace=True)

# Apply the clean_text function to preprocess the 'content' column
df['content'] = df['content'].apply(clean_text)


#The column contained unnecessary words like recorded reports or authors name, so remove these by removing everything from start to the first colon (:)
df['content'] = df['content'].str.split(':', n=1).str[-1]

#Remove the text 'copyright' with year present in some articles
df['content'] = df['content'].str.replace('Copyright Business Recorder, \d{4}\.', '', regex=True)

# Define a regular expression pattern to match lines with timestamps and program titles
pattern = r'^\d{2}:\d{2}\s[A-Z ]+'

#Remove the rows with matching above pattern
df = df[~df['content'].str.match(pattern)]

#Path for output csv file
output = '/content/drive/My Drive/clean_news_data.csv'

# Open the output CSV file for writing. This piece of code is for eliminating rows with less than 10 words.
with open(output, 'w', newline='', encoding='utf-8') as output_file:
    writer = csv.writer(output_file)

    # Iterate through rows in the DataFrame
    for index, row in df.iterrows():
        # Write rows with 10 or more words in the 'content' column to the output file
        if len(row['content'].split()) >= 10:
            writer.writerow([row['content']])

print("Preprocessed data saved to:", output)


Preprocessed data saved to: /content/drive/My Drive/clean_news_data.csv


# Preprocessing of reviews_only file



In [ ]:
import pandas as pd
import re

# Function to clean the text
def clean_text(text):
    #Remove everything except alphabets, integers, commas, and full stops.
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,]', '', text)
    # Removing extra full stops, question marks and commas and replacing them with single full stop, question mark and comma
    cleaned_text = re.sub(r'\.{2,}', '.', cleaned_text)
    cleaned_text = re.sub(r'\?{2,}', '?', cleaned_text)
    cleaned_text = re.sub(r'\,{2,}', ',' , cleaned_text)
    return cleaned_text.strip()

#Path of input csv file
file_path = '/content/drive/MyDrive/reviews_only.csv'

#Read csv file in df
df = pd.read_csv(file_path)

# Remove empty rows
df.dropna(inplace=True)

# Remove rows with less than 10 words
df['Word Count'] = df['content'].apply(lambda x: len(x.split()))
df = df[df['Word Count'] >= 10]

# Clean the review text
df['Cleaned Review'] = df['content'].apply(clean_text)

# Drop the 'Word Count' column as it's no longer needed
df.drop(columns=['Word Count'], inplace=True)

# Remove duplicate rows based on the 'Cleaned Review' column
df.drop_duplicates(subset=['Cleaned Review'], inplace=True)

# extract the cleaned review text
cleaned_reviews = df['Cleaned Review']
print(cleaned_reviews)

# Save the cleaned reviews to a separate CSV file
cleaned_reviews.to_csv('/content/drive/MyDrive/cleaned_reviews.csv', index=False)


0       1. Exterior and interior both are out class.2....
1       eliability is a real strong point for the Coro...
2       Grande is the best car  its fuel economy is be...
3       Toyota Corolla 1982 GLexterior of this car is ...
4       Exterior of car is awesome.Pick up and engine ...
                              ...                        
9495    Hello guys, my experience with Honda CG 125 ha...
9496    this bike best but please launch ybr 125 Custo...
9497    The look and feel of the the bike is good, the...
9498    First of all look is good decent and stylish b...
9499    Lorem Ipsum is simply dummy text of the printi...
Name: Cleaned Review, Length: 4877, dtype: object


# Joining both files

In [ ]:
import pandas as pd

# Load both files into DataFrames
file1_path = '/content/drive/MyDrive/clean_news_data_.csv'
file2_path = '/content/drive/MyDrive/cleaned_reviews.csv'
news_df = pd.read_csv(file1_path)
reviews_df = pd.read_csv(file2_path)

# Concatenate both df
merged_df = pd.concat([news_df, reviews_df], ignore_index=True)

# Save the concatenated df to a new file
output = '/content/drive/My Drive/complete_file.csv'
merged_df.to_csv(output, index=False)

print("Merged file saved to:", output)


Merged file saved to: /content/drive/My Drive/complete_file.csv


# LLM

In [ ]:
#installing necessary libraries
!pip install transformers

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
#The Model used for this task is TheBloke/Mistral-7B-Instruct-v0.1-GGUF.

In [ ]:
import csv
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os

# Loading Language model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        model_type="mistral",
        max_new_tokens=1048,
        temperature=0.3
    )
    return llm

# File Processing
def file_processing(file_path):
    # Load data from CSV
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = ""
        for row in reader:
            data += row[0]

    question_gen = data

    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    return document_ques_gen

# Prompt Engineering
def llm_pipeline(file_path):
    document_ques_gen = file_processing(file_path)
    llm_ques_gen_pipeline = load_llm()

    prompt_template = """
    You are an expert at creating questions based on text data.
    Your goal is to prepare students for test and exams.
    You do this by asking questions about the text below:

    ------------
    {text}
    ------------

    Create questions that will help customers know about the automobile.
    Make sure not to lose any important information.

    QUESTIONS:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

    refine_template = ("""
    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: {existing_answer}.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    {text}
    ------------

    Given the new context, refine the original questions in English.
    If the context is not helpful, please provide the original questions.
    QUESTIONS:
    """
    )

    REFINE_PROMPT_QUESTIONS = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    ques_gen_chain = load_summarize_chain(llm=llm_ques_gen_pipeline,
                                          chain_type="refine",
                                          verbose=True,
                                          question_prompt=PROMPT_QUESTIONS,
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

    ques = ques_gen_chain.run(document_ques_gen)

    llm_answer_gen = load_llm()

    ques_list = ques.split("\n")
    filtered_ques_list = [element for element in ques_list if element.endswith('?') or element.endswith('.')]

    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                          chain_type="stuff")

    return answer_generation_chain, filtered_ques_list

# Generating CSV file in the drive
def get_csv(file_path):
    answer_generation_chain, ques_list = llm_pipeline(file_path)
    base_folder = 'output/'
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
    output_file = os.path.join(base_folder, "QA.csv")
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Question", "Answer"])

        for question in ques_list:
            print("Question: ", question)
            answer = answer_generation_chain.run(question)
            print("Answer: ", answer)
            print("--------------------------------------------------\n\n")

            # Save answer to CSV file
            csv_writer.writerow([question, answer])
    return output_file

csv_file_path = "/content/drive/MyDrive/complete_file.csv"
output_file = get_csv(csv_file_path)
print("CSV file generated:", output_file)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q2_K.gguf:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

...


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: 1. What is the exterior and interior finish of the Toyota Corolla Grande?
    2. What are the key features of the Toyota Corolla Grande?
    3. How would you rate the suspension system in the Toyota Corolla Grande? 
    4. What is the performance of the Toyota Corolla Grande? 
    5. How does the Toyota Corolla Grande handle driving on motorways or GT roads? 
    6. Would you say that the Toyota Corolla Grande is reliable? 
    7. What are some of the features that make the Toyota Corolla Toyota Toyota Toyota Corolla Toyota Corolla Toyota Corolla.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    beautiful  Toyota Gr


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: 1. What is the exterior finish of the Toyota Corolla Grande?
    2. What are the key features of the Toyota Corolla Grande?
    3. How would you rate the suspension system in the Toyota Corolla Grande? 
    4. What is the performance of the Toyota Corolla Grande? 
    5. How does the Toyota Corolla Grande handle driving on motorways or GT roads with highways?
GT roads? GT roads?.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    performance is matchless. Most powerful than its competitors like honda. Ground clearance is good. Fuel average is very good. Suspension awesome. Price is on higher side which should be less.


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: 1. What is the exterior finish of the Toyota Corolla Grande?
    2. What are the key features of the Toyota Corolla Grande?
    3. How would you rate the suspension system in the Toyota Corolla Grande? 
    4. What is the performance of the Toyota Corolla Grande? 
    5. How does the Toyota Corolla Grande handle driving on motorways or GT roads with highways?.

    REFINE QUESTIONS:
    1. What would refinex. What are there were the car in 1. What is the exterior finish of the exterior finish of the suspension system.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    AVERAGE IT GIVES IS 12 TO 14 IN CITY AND MORE IN H


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: 1. What are the exterior features of Toyota Altis?
    2. What are the key features of Toyota Altis?
    3. How would you rate the suspension system of the Toyota Altis?
    4. Can you tell me about the performance of Toyota Altis?
    5. How does the Toyota Altis handle driving on motorways or GT roads with highways?
    REFINED QUESTIONS:
    1. What are some notable exterior features of the Toyota Altis, such as its design and trunk capacity?
    2. Can you describe the key features of the Toyota Altis, including its comfort, durability, reliability, and market price?
    3. How would you rate the suspension system of the Toyota Altis, based on the customer feedback provided in the context?
    4. According to


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating practice questions based on text data.
    Your goal is to prepare the student for exam and test.
    We have received some practice questions to a certain extent: 1. What are some notable exterior features of the notable exterior features that are notable exterior features that the notable exterior design and the key exterior features of the notable external characteristics of the most prominent exterior features of the notable exterior features of the notable exterior features of the noticeable exterior features of the notable exterior features of the notable exterior features that you.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    feul aver 13.5 in city n 15 on longs. good handling.fully satisfied to have it.Totally Comfortable and having excellent fuel average and a rel

In [ ]:
pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
